In [1]:
import numpy as np
import pandas as pd

In [3]:
import requests
import os
import json
from tqdm import tqdm

In [6]:
import multiprocessing

In [7]:
from functools import partial

In [8]:
n_cpus = multiprocessing.cpu_count()

allArtworks.jsonl came from [this repo](https://github.com/art-institute-of-chicago/api-data)
It has id for artworks which can be used to get more fields (ingluding image id) for the artworks using [ARTIC API](https://api.artic.edu/docs/#iiif-image-api)

In [9]:
with open("allArtworks.jsonl", 'r') as json_file:
    all_artworks_json = list(json_file)

for json_str in all_artworks_json:
    result = json.loads(json_str)
    print(f"result: {result}")
    print(isinstance(result, dict))
    break

result: {'id': 4, 'title': 'Priest and Boy', 'main_reference_number': '1880.1', 'department_title': 'Prints and Drawings', 'artist_title': 'Lawrence Carmichael Earle'}
True


In [10]:
len(all_artworks_json)

122435

In [11]:
def single_process(start_id, n_processes=n_cpus):
    for i in tqdm(range(start_id, len(all_artworks_json), n_processes)):
        art_url = "https://api.artic.edu/api/v1/artworks/{}".format(json.loads(all_artworks_json[i])["id"])
        response = requests.get(art_url)
        art = json.loads(response.content)
        with open("all_artworks_api_response_data_{}.jsonl".format(start_id), "a") as f:
            f.write(json.dumps(art))
            f.write("\n")

In [15]:
parallel_processes = []
for i in range(n_cpus):
    p = multiprocessing.Process(target=partial(single_process, i))
    parallel_processes.append(p)

In [16]:
for proc in parallel_processes:
    proc.start()

  6%|▌         | 1724/30608 [28:41<7:10:13,  1.12it/s] 